# HOC Mass Email 

_Note: advisement list downloaded at 5:59 EST on December 6th, 2017_

In [1]:
import pandas as pd
import numpy as np
pd.options.display.max_rows = 1000
df = pd.read_csv('advisement_list.csv')


df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,NaN,Teacher,Room Number,Name,Email Address
1,NaN,Kempf,E121,Gavin Arnett,gmanarnett@gmail.com
2,NaN,Dodd,PE07,Ishan Bapat,ibapat02@gmail.com
3,NaN,Hannaford,G110,Bhavik Singh,bhavik244@gmail.com
4,NaN,Mantegna,B103,Matt Aubley,mattaubley@gmail.com


In [2]:
## clean up dataframe

df = df.drop('Unnamed: 0', axis=1)
df.columns = ['Teacher', 'Room Number', 'Name', 'Email Address']
df = df.drop(0)

df.head()

,Teacher,Room Number,Name,Email Address
1,Kempf,E121,Gavin Arnett,gmanarnett@gmail.com
2,Dodd,PE07,Ishan Bapat,ibapat02@gmail.com
3,Hannaford,G110,Bhavik Singh,bhavik244@gmail.com
4,Mantegna,B103,Matt Aubley,mattaubley@gmail.com
5,Foskey,H114,Jeremy Maier,jeremy.maier5@gmail.com


In [3]:
## look for anomalies

df.iloc[65]

Teacher             Chrzanowski
Room Number                F108
Name             Chase Williams
Email Address               NaN
Name: 66, dtype: object

In [4]:
df[df['Email Address'].isnull()]

,Teacher,Room Number,Name,Email Address
14,Bell,E120,Hamilton,NaN
16,Ballard,PRT6,Nic,NaN
17,Pettay,G116,Alex,NaN
18,Pham,G113,Tyler,NaN
23,"Miller, S",F119,Cole,NaN
27,Fairchild,PRT1,Raul & Jose,NaN
36,Collins,PRT3,Jonathan Tarrant,NaN
45,"Kolodziej, K",G104,Sophia,NaN
56,Chiclana,H116,Zenobia Sarkari (will send Ms. Bell to your a...,NaN
66,Chrzanowski,F108,Chase Williams,NaN


In [5]:
to_email = df[df['Email Address'].notnull()]

In [6]:
delimiters = [',', ';']
def has_delimiter(string):
    return any([string.find(delimiter) != -1 for delimiter in delimiters])

def split(string):
    indices = []
    elements = []
    for delimiter in delimiters:
        substring = string
        while substring.find(delimiter) != -1:
            index = substring.find(delimiter)
            indices.append(index)
            substring = substring[index + 1:]
    indices.sort()
    elements.append(string[:indices[0]])
    for i, start_index in enumerate(indices):
        if (i + 1 == len(indices)):
            elements.append(string[start_index + 1:])
            break
        else:
            stop_index = indices[i + 1]
            elements.append(string[start_index + 1:stop_index])
    return elements
                
def get_people(row):
    names = []
    if has_delimiter(row[2]):
        names = split(row[2])
    else:
        return [row]
    
    email_addresses = split(row[3])
    
    people = []
    for name, email in zip(names, email_addresses):
        people.append([row[0], row[1], name.strip(), email.strip()])
    return people


In [7]:
df[df['Email Address'].isnull()]

,Teacher,Room Number,Name,Email Address
14,Bell,E120,Hamilton,NaN
16,Ballard,PRT6,Nic,NaN
17,Pettay,G116,Alex,NaN
18,Pham,G113,Tyler,NaN
23,"Miller, S",F119,Cole,NaN
27,Fairchild,PRT1,Raul & Jose,NaN
36,Collins,PRT3,Jonathan Tarrant,NaN
45,"Kolodziej, K",G104,Sophia,NaN
56,Chiclana,H116,Zenobia Sarkari (will send Ms. Bell to your a...,NaN
66,Chrzanowski,F108,Chase Williams,NaN


In [8]:
body_template = "<html><head></head><body>{name}, you are finalized to help facilitate in " \
    "{teacher}'s room ({room_number})." \
    "<br><br>Here are links to the presentations should you need them" \
    "<br>Lightbot: {lightbot}" \
    "<br>Scratch: {scratch}" \
    "<br><br>If you have any questions, please do bring them to Mr. Lee or Mrs. Deepa." \
    "<br><br>Stay posted for additional details.</body></html>"

scratch_presentation_link = 'https://drive.google.com/open?id=1aijdEIxEvNJaU6cCkydfzEoN1FTh_z3nii5JyZrQ0Ac'
lightbot_presentation_link = 'https://drive.google.com/open?id=1Y_bxcjgpepkc5UlMpT1f-RrmGdiS6ins2vh2Q5WwroU'

In [9]:
arr = [
    ('some other teacher', 'another room in the school', 'an idiot named henna, an idiot named shruti', 'bornofawesomeness@gmail.com, doaasdfsd')
]
test_emails = pd.DataFrame(arr)
test_emails.columns = df.columns
test_emails

,Teacher,Room Number,Name,Email Address
0,some other teacher,another room in the school,"an idiot named henna, an idiot named shruti","bornofawesomeness@gmail.com, doaasdfsd"


In [10]:
import helper
from helper import send_emails
from smtplib import SMTPRecipientsRefused

data = []
for index, row in to_email.iterrows():
    people = get_people(row)
    for person in people:
        body_details = {
            'name': person[2],
            'teacher': person[0],
            'room_number': person[1],
            'lightbot': lightbot_presentation_link,
            'scratch': scratch_presentation_link
        }
        body = body_template.format(**body_details)
        email = person[3]
        data.append(['Hour of test', body, email])
send_emails(data)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
l.com
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
